# ResNet 50 (Transfer Learning)

In [1]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.callbacks import ReduceLROnPlateau
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from sklearn.model_selection import train_test_split
import shutil

In [2]:
# print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [6]:
# Create directories for training and validation data
def setup_directories(base_dir, images_dir, csv_path):
    train_dir = os.path.join(base_dir, 'train')
    val_dir = os.path.join(base_dir, 'val')
    for dir_path in [train_dir, val_dir]:
        os.makedirs(os.path.join(dir_path, 'positive'), exist_ok=True)
        os.makedirs(os.path.join(dir_path, 'negative'), exist_ok=True)

    # Split data into training and validation
    df = pd.read_csv(csv_path)
    train_df, val_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['ground_truth'])
    move_images(train_df, images_dir, train_dir)
    move_images(val_df, images_dir, val_dir)
    return train_dir, val_dir

def move_images(dataframe, src_dir, dest_dir):
    for _, row in dataframe.iterrows():
        src_path = os.path.join(src_dir, f"{row['id']}.png")
        label_dir = 'positive' if row['ground_truth'] == 1 else 'negative'
        dst_path = os.path.join(dest_dir, label_dir, f"{row['id']}.png")
        shutil.copy(src_path, dst_path)

In [7]:
# Initialize and compile the model
def initialize_model():
    resnet_weights_path = 'pre_trained_models/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'
    model = Sequential([
        ResNet50(include_top=False, pooling='avg', weights=resnet_weights_path),
        Dense(2, activation='softmax')
    ])
    model.layers[0].trainable = False  # Freeze the ResNet50 pre-trained layers
    model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])
    model.summary()
    return model

# Set up image data generators
def create_generators(base_temp_dir, image_size):
    data_generator = ImageDataGenerator(preprocessing_function=preprocess_input)
    train_generator = data_generator.flow_from_directory(
        os.path.join(base_temp_dir, 'train'),
        target_size=(image_size, image_size),
        batch_size=12,
        class_mode='categorical'
    )
    validation_generator = data_generator.flow_from_directory(
        os.path.join(base_temp_dir, 'val'),
        target_size=(image_size, image_size),
        batch_size=20,
        class_mode='categorical'
    )
    return train_generator, validation_generator

# Evaluate the model
def evaluate_model(model, test_images, test_labels):
    predictions = model.predict(test_images)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(test_labels, axis=1)
    
    accuracy = accuracy_score(true_classes, predicted_classes)
    precision = precision_score(true_classes, predicted_classes, average='macro')
    recall = recall_score(true_classes, predicted_classes, average='macro')
    f1 = f1_score(true_classes, predicted_classes, average='macro')
    
    print(f"Accuracy: {accuracy:.2f}")
    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")
    print(f"F1 Score: {f1:.2f}")
    
    # Display confusion matrix
    cm = confusion_matrix(true_classes, predicted_classes)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    disp.plot(cmap=plt.cm.Blues)
    plt.title('Confusion Matrix')
    plt.show()

def load_and_prepare_image(image_id, directory, target_size):
    """
    Load an image file, resize it, and preprocess it for ResNet50.

    Args:
    image_id (str): The identifier for the image file, usually the filename.
    directory (str): The directory path where the image file is stored.
    target_size (tuple): The target image size (height, width) as expected by the model.

    Returns:
    numpy.ndarray: The preprocessed image array suitable for model prediction.
    """
    image_path = os.path.join(directory, f'{image_id}.png')
    image = load_img(image_path, target_size=target_size)
    image_array = img_to_array(image)
    image_array = preprocess_input(image_array)  # Apply ResNet50 preprocessing
    return image_array



In [8]:
# Main execution function
def main():
    base_temp_dir = 'data/resnet50_output'
    images_dir = 'data/images_train'
    train_csv_path = 'data/train.csv'
    test_csv_path = 'data/test.csv'
    images_test_dir = 'data/images_test'

    model = initialize_model()
    train_dir, val_dir = setup_directories(base_temp_dir, images_dir, train_csv_path)
    train_generator, validation_generator = create_generators(base_temp_dir, 224)

    # Train the model
    lr_reduction = ReduceLROnPlateau(monitor='val_loss', patience=2, verbose=1, factor=0.5, min_lr=1e-6)
    history = model.fit(train_generator, epochs=10, validation_data=validation_generator, callbacks=[lr_reduction])

    # Prepare test data
    test_df = pd.read_csv(test_csv_path)
    test_images = [load_and_prepare_image(x, images_test_dir, (539, 528)) for x in test_df['id']]
    test_images_array = np.array(test_images)
    test_labels = pd.get_dummies(test_df['ground_truth']).values

    # Evaluate the model
    evaluate_model(model, test_images_array, test_labels)

if __name__ == "__main__":
    main()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 dense (Dense)               (None, 2)                 4098      
                                                                 
Total params: 23591810 (90.00 MB)
Trainable params: 4098 (16.01 KB)
Non-trainable params: 23587712 (89.98 MB)
_________________________________________________________________
Found 2720 images belonging to 2 classes.
Found 681 images belonging to 2 classes.
Epoch 1/10
227/227 [==============================] - 79s 340ms/step - loss: 1.5674 - accuracy: 0.7074 - val_loss: 0.6139 - val_accuracy: 0.8062 - lr: 0.0100
Epoch 2/10
227/227 [==============================] - 77s 338ms/step - loss: 1.3679 - accuracy: 0.7360 - val_loss: 0.5964 - val_accuracy: 0.8076 - lr: 0.0100
Epoch 3/1

KeyboardInterrupt: 